## Numpy Implementation

In [1]:
import numpy as np

In [2]:
def calculate_inception_score(p_yx, eps=1e-16):
  p_y = np.expand_dims(p_yx.mean(axis=0), 0)
  kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
  sum_kl_d = kl_d.sum(axis=1)
  avg_kl_d = np.mean(sum_kl_d)
  is_score = np.exp(avg_kl_d)
  return is_score

# Conditional probabilities for high quality images 
# (e.g the model has correctly predicted that the input
# feature vectors belong to that particular class.)
p_yx = np.asarray([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
score = calculate_inception_score(p_yx)
score

2.999999999999999

In [3]:
# Conditional probabilities for low quality images
# (e.g the model is confused in predicting the input
# feature vector belongs to any particular class and 
# gave all the classes an equal probabilty.)
p_yx = np.asarray([[0.33, 0.33, 0.33], [0.33, 0.33, 0.33], [0.33, 0.33, 0.33]]) 
score = calculate_inception_score(p_yx)
score

1.0

## Keras Implementation

In [4]:
from tensorflow import keras

In [5]:
def calculate_inception_score(images, n_split=10, eps=1E-16): 
  # load inception v3 model 
  model = keras.applications.inception_v3.InceptionV3() 
  # convert from uint8 to float32 
  processed = images.astype('float32') 
  # pre-process raw images for inception v3 model 
  processed = keras.applications.inception_v3.preprocess_input(processed) 
  # predict class probabilities for images 
  yhat = model.predict(processed) 
  print(yhat)
  # enumerate splits of images/predictions 
  scores = list() 
  n_part = np.floor(images.shape[0] / n_split) 
  for i in range(n_split): 
    # retrieve p(y|x) 
    ix_start, ix_end = i * n_part, i * n_part + n_part 
    p_yx = yhat[int(ix_start):int(ix_end)] 
    # calculate p(y) 
    p_y = np.expand_dims(p_yx.mean(axis=0), 0) 
    # calculate KL divergence using log probabilities 
    kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps)) 
    # sum over classes 
    sum_kl_d = kl_d.sum(axis=1) 
    # average over images 
    avg_kl_d = np.mean(sum_kl_d) 
    # undo the log 
    is_score = np.exp(avg_kl_d) # store
  scores.append(is_score)
  is_avg, is_std = np.mean(scores), np.std(scores)
  return is_avg, is_std

images = np.ones((50, 299, 299, 3))
print("Loaded: ", images.shape)
is_avg, is_std = calculate_inception_score(images)
print("Score: ", is_avg, is_std)

Loaded:  (50, 299, 299, 3)
2/2 [==============================] - 13s 2s/step
[[4.2931549e-04 7.8431319e-04 3.2570600e-04 ... 9.4141054e-05
  5.5545609e-04 1.0611054e-03]
 [4.2931549e-04 7.8431319e-04 3.2570600e-04 ... 9.4141054e-05
  5.5545609e-04 1.0611054e-03]
 [4.2931549e-04 7.8431319e-04 3.2570600e-04 ... 9.4141054e-05
  5.5545609e-04 1.0611054e-03]
 ...
 [4.2931590e-04 7.8430498e-04 3.2570196e-04 ... 9.4140429e-05
  5.5545318e-04 1.0610998e-03]
 [4.2931590e-04 7.8430498e-04 3.2570196e-04 ... 9.4140429e-05
  5.5545318e-04 1.0610998e-03]
 [4.2931590e-04 7.8430498e-04 3.2570196e-04 ... 9.4140429e-05
  5.5545318e-04 1.0610998e-03]]
Score:  1.0 0.0


In [6]:
import skimage as skimg 
print(skimg.__version__)

0.19.3


In [ ]:
def scale_images(images, new_shape):
  images_list = list()
  for image in images:
    new_image = skimg.transform.resize(image, new_shape, 0)
    images_list.append(new_image)
  return np.asarray(images_list)

def calculate_inception_score(images, n_split=10, eps=1e-16):
  model = keras.applications.inception_v3.InceptionV3()
  scores = list()
  n_part = np.floor(images.shape[0] / n_split)
  for i in range(n_split):
    ix_start, ix_end = i * n_part, (i+1) * n_part
    subset = images[int(ix_start):int(ix_end)]
    subset = subset.astype('float32')
    subset = scale_images(subset, (299, 299, 3))
    subset = keras.applications.inception_v3.preprocess_input(subset)
    p_yx = model.predict(subset)
    p_y = np.expand_dims(p_yx.mean(axis=0), 0)
    kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
    sum_kl_d = kl_d.sum(axis=1)
    avg_kl_d = np.mean(sum_kl_d)
    is_score = np.exp(avg_kl_d)
    scores.append(is_score)
  is_avg, is_std = np.mean(scores), np.std(scores)
  return is_avg, is_std

(images, _), (_, _) = keras.datasets.cifar10.load_data()
np.random.shuffle(images)
print('Loaded: ', images.shape)
is_avg, is_std = calculate_inception_score(images) 
print('score', is_avg, is_std)

Loaded:  (50000, 32, 32, 3)
